In [ ]:
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import datetime
import shutil
from pathlib import Path
import trackpy
from iris.time import PartialDateTime
import cartopy.crs as ccrs
import xarray as xr
import netCDF4 as nc
import scipy
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure

#import packages that output memory usage:
#from sys import getsizeof

import tobac #tobac package cloned from https://github.com/tobac-project/tobac.git

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import mpl_toolkits
from mpl_toolkits.basemap import Basemap

In [ ]:
## testing on 1 year of precip data from CPM-PD ##
filename_2001 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2001.nc'
filename_2002 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2002.nc'
filename_2003 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2003.nc'
filename_2004 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2004.nc'
filename_2005 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2005.nc'
filename_2006 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2006.nc'
filename_2007 = '/scratch/hgilmour/cpm_HINDCAST/total_precip/yearly_files/regridded/regridded_total_precip_2007.nc'


precip_2001 = xr.open_dataset(filename_2001)
precip_2001 = precip_2001.unknown

precip_2002 = xr.open_dataset(filename_2002)
precip_2002 = precip_2002.unknown

precip_2003 = xr.open_dataset(filename_2003)
precip_2003 = precip_2003.unknown

precip_2004 = xr.open_dataset(filename_2004)
precip_2004 = precip_2004.unknown

precip_2005 = xr.open_dataset(filename_2005)
precip_2005 = precip_2005.unknown

precip_2006 = xr.open_dataset(filename_2006)
precip_2006 = precip_2006.unknown

precip_2007 = xr.open_dataset(filename_2007)
precip_2007 = precip_2007.unknown

In [ ]:
### SAME AS ABOVE BUT WITH ONLY MCS RAINFALL FROM MASKED AREAS ###

## testing on 1 year of precip data from CPM-PD ##
filename_2001 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2001.nc'
filename_2002 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2002.nc'
filename_2003 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2003.nc'
filename_2004 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2004.nc'
filename_2005 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2005.nc'
filename_2006 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2006.nc'
filename_2007 = '/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/mcs_rainfall_HINDCAST_2007.nc'


precip_2001_mcs = xr.open_dataset(filename_2001)
precip_2001_mcs = precip_2001_mcs.unknown

precip_2002_mcs = xr.open_dataset(filename_2002)
precip_2002_mcs = precip_2002_mcs.unknown

precip_2003_mcs = xr.open_dataset(filename_2003)
precip_2003_mcs = precip_2003_mcs.unknown

precip_2004_mcs = xr.open_dataset(filename_2004)
precip_2004_mcs = precip_2004_mcs.unknown

precip_2005_mcs = xr.open_dataset(filename_2005)
precip_2005_mcs = precip_2005_mcs.unknown

precip_2006_mcs = xr.open_dataset(filename_2006)
precip_2006_mcs = precip_2006_mcs.unknown

precip_2007_mcs = xr.open_dataset(filename_2007)
precip_2007_mcs = precip_2007_mcs.unknown

In [ ]:
precip_2001_mcs_fillna = precip_2001_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 

precip_2002_mcs_fillna = precip_2002_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 

precip_2003_mcs_fillna = precip_2003_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 

precip_2004_mcs_fillna = precip_2004_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 

In [ ]:
precip_2005_mcs_fillna = precip_2005_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 
precip_2005_mcs_fillna

precip_2006_mcs_fillna = precip_2006_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 
precip_2006_mcs_fillna

precip_2007_mcs_fillna = precip_2007_mcs.fillna(0) ## filling all nan values in the mask precip dataset with 0 
precip_2007_mcs_fillna

In [ ]:
# Wrap it into a simple function
def season_mean(ds):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = (
        month_length.groupby("time.season") / month_length.groupby("time.season").sum()
    )

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby("time.season").sum().values, np.ones(4))

    # Calculate the weighted average
    return (ds * weights).groupby("time.season").sum(dim="time")

In [ ]:
precip_2001_mcs_weighted = season_mean(precip_2001_mcs)
precip_2002_mcs_weighted = season_mean(precip_2002_mcs)
precip_2003_mcs_weighted = season_mean(precip_2003_mcs)
precip_2004_mcs_weighted = season_mean(precip_2004_mcs)
precip_2005_mcs_weighted = season_mean(precip_2005_mcs)
precip_2006_mcs_weighted = season_mean(precip_2006_mcs)
precip_2007_mcs_weighted = season_mean(precip_2007_mcs)


In [ ]:
precip_2001_weighted = season_mean(precip_2001)
precip_2002_weighted = season_mean(precip_2002)
precip_2003_weighted = season_mean(precip_2003)
precip_2004_weighted = season_mean(precip_2004)
precip_2005_weighted = season_mean(precip_2005)
precip_2006_weighted = season_mean(precip_2006)
precip_2007_weighted = season_mean(precip_2007)

In [ ]:
percentage_2001 = ((precip_2001_mcs_weighted) / (precip_2001_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall
percentage_2002 = ((precip_2002_mcs_weighted) / (precip_2002_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall
percentage_2003 = ((precip_2003_mcs_weighted) / (precip_2003_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall
percentage_2004 = ((precip_2004_mcs_weighted) / (precip_2004_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall
percentage_2005 = ((precip_2005_mcs_weighted) / (precip_2005_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall
percentage_2006 = ((precip_2006_mcs_weighted) / (precip_2006_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall
percentage_2007 = ((precip_2007_mcs_weighted) / (precip_2007_weighted))*100 ## calculating the percentage of mcs rainfall to total rainfall

In [ ]:
percentage_2006

In [ ]:
percentage_all = xr.concat([percentage_2001, percentage_2002, percentage_2003, percentage_2004, percentage_2005, percentage_2006, percentage_2007], dim='time', compat='override', coords='all')

mean_percentage_all = percentage_all.mean(dim='time')

mean_percentage_all.to_netcdf('/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/seasonal_percentages_HINDCAST.nc')

**Start from here if seasonal percentages files have already been made**

In [ ]:
mean_percentage_all_PD = xr.open_dataset('/project/cssp_brazil/mcs_tracking_HG/CPM_PD_TRACKS/mcs_rainfall/seasonal_percentages_PD.nc')
mean_percentage_all_HINDCAST = xr.open_dataset('/project/cssp_brazil/mcs_tracking_HG/CPM_HINDCAST_TRACKS/mcs_rainfall/seasonal_percentages_HINDCAST.nc')


In [ ]:
mean_percentage_all_PD = mean_percentage_all_PD.__xarray_dataarray_variable__
mean_percentage_all_HINDCAST = mean_percentage_all_HINDCAST.__xarray_dataarray_variable__
difference = mean_percentage_all_PD - mean_percentage_all_HINDCAST

In [ ]:
# Quick plot to show the results (https://docs.xarray.dev/en/stable/examples/monthly-means.html)

# Irregular levels to illustrate the use of a proportional colorbar
levels = [0, 10, 20, 30, 40, 50, 60, 70]
levels_diff = [-30, -20, -10, 0, 10, 20, 30]

fig, geo_axes = plt.subplots(nrows=4, ncols=3, figsize=(14, 12), subplot_kw={'projection': ccrs.PlateCarree()})
for i, season in enumerate(("DJF", "MAM", "JJA", "SON")):
    m = Basemap(llcrnrlon=-85, llcrnrlat=-40,urcrnrlon=-30, urcrnrlat=15) # create Basemap object
    m.drawcoastlines(linewidth=1, ax=geo_axes[i,0]) # draw coastlines
    mean_percentage_all_PD.sel(season=season).plot.pcolormesh(
        ax=geo_axes[i, 0],
        vmin=0,
        vmax=70,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="max",
        cbar_kwargs={'ticks': levels, 'label': 'precipitation from MCSs [%]'}
    )
    
    m = Basemap(llcrnrlon=-85, llcrnrlat=-40,urcrnrlon=-30, urcrnrlat=15) # create Basemap object
    m.drawcoastlines(linewidth=1, ax=geo_axes[i,1]) # draw coastlines
    mean_percentage_all_HINDCAST.sel(season=season).plot.pcolormesh(
        ax=geo_axes[i, 1],
        vmin=0,
        vmax=70,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="max",
        cbar_kwargs={'ticks': levels, 'label': 'precipitation from MCSs [%]'}
    )
    
    m = Basemap(llcrnrlon=-85, llcrnrlat=-40,urcrnrlon=-30, urcrnrlat=15) # create Basemap object
    m.drawcoastlines(linewidth=1, ax=geo_axes[i,2]) # draw coastlines
    difference.sel(season=season).plot.pcolormesh(
        ax=geo_axes[i, 2],
        vmin=-30,
        vmax=30,
        cmap="RdBu",
        add_colorbar=True,
        extend="both",
        cbar_kwargs={'ticks': levels_diff, 'label': 'precipitation from MCSs [%]'}
    )
    

    geo_axes[i, 0].set_ylabel(season, fontsize=18)
    geo_axes[i, 1].set_ylabel(" ")
    geo_axes[i, 2].set_ylabel(" ")
    
for ax in geo_axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.axis("tight")
    ax.set_xlabel("")

geo_axes[0, 0].set_title("CPM-PD", fontsize=18)
geo_axes[0, 1].set_title("CPM-Hindcast", fontsize=18)
geo_axes[0, 2].set_title("Difference", fontsize=18)
geo_axes[1, 0].set_title(" ")
geo_axes[1, 1].set_title(" ")
geo_axes[1, 2].set_title(" ")
geo_axes[2, 0].set_title(" ")
geo_axes[2, 1].set_title(" ")
geo_axes[2, 2].set_title(" ")
geo_axes[3, 0].set_title(" ")
geo_axes[3, 1].set_title(" ")
geo_axes[3, 2].set_title(" ")


geo_axes[0, 1].set_xlim(-85, -30)
geo_axes[1, 1].set_xlim(-85, -30)
geo_axes[2, 1].set_xlim(-85, -30)
geo_axes[3, 1].set_xlim(-85, -30)

geo_axes[0, 2].set_xlim(-85, -30)
geo_axes[1, 2].set_xlim(-85, -30)
geo_axes[2, 2].set_xlim(-85, -30)
geo_axes[3, 2].set_xlim(-85, -30)


plt.tight_layout()

fig.suptitle("MCS contribution to total rainfall", fontsize=16, y=1.02)

In [ ]:
min_lon = -72
min_lat = -35
max_lon = -52 
max_lat = -18 


percentage_all_lpb = percentage_all.sel(latitude=slice(min_lat,max_lat), longitude=slice(min_lon,max_lon)).mean(dim='time')
percentage_all_lpb

In [ ]:
percentage_all_lpb.sel().mean()

In [ ]:
# Quick plot to show the results (https://docs.xarray.dev/en/stable/examples/monthly-means.html)

# Irregular levels to illustrate the use of a proportional colorbar
levels = [0, 10, 20, 30, 40, 50, 60, 70]

fig, geo_axes = plt.subplots(nrows=4, ncols=3, figsize=(14, 12), subplot_kw={'projection': ccrs.PlateCarree()})
for i, season in enumerate(("DJF", "MAM", "JJA", "SON")):
    m = Basemap(llcrnrlon=-85, llcrnrlat=-40,urcrnrlon=-30, urcrnrlat=15) # create Basemap object
    m.drawcoastlines(linewidth=1, ax=geo_axes[i,0]) # draw coastlines
    percentage_all_lpb.sel(season=season).plot.pcolormesh(
        ax=geo_axes[i, 0],
        vmin=0,
        vmax=70,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="max",
        cbar_kwargs={'ticks': levels, 'label': 'precipitation from MCSs [%]'}
    )

    geo_axes[i, 0].set_ylabel(season)
    geo_axes[i, 1].set_ylabel(" ")
    geo_axes[i, 2].set_ylabel(" ")

for ax in geo_axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.axis("tight")
    ax.set_xlabel("")

geo_axes[0, 0].set_title("CPM-PD")
geo_axes[0, 1].set_title("CPM-Hindcast")
geo_axes[0, 2].set_title("Difference")

plt.tight_layout()

fig.suptitle("MCS contribution to total rainfall", fontsize=16, y=1.02)

In [ ]:
precip_2002_mcs_fillna = precip_2002_mcs.fillna(0)

cumulative_precip_2002_mcs = precip_2002_mcs_fillna.sum(dim='time', skipna=True)

cumulative_precip_2002 = precip_2002.sum(dim='time', skipna=True)

percentage_2002 = ((cumulative_precip_2002_mcs) / (cumulative_precip_2002))*100

In [ ]:
precip_2003_mcs_fillna = precip_2003_mcs.fillna(0)

cumulative_precip_2003_mcs = precip_2003_mcs_fillna.sum(dim='time', skipna=True)

cumulative_precip_2003 = precip_2003.sum(dim='time', skipna=True)

percentage_2003 = ((cumulative_precip_2003_mcs) / (cumulative_precip_2003))*100

In [ ]:
precip_2004_mcs_fillna = precip_2004_mcs.fillna(0)

cumulative_precip_2004_mcs = precip_2004_mcs_fillna.sum(dim='time', skipna=True)

cumulative_precip_2004 = precip_2004.sum(dim='time', skipna=True)

percentage_2004 = ((cumulative_precip_2004_mcs) / (cumulative_precip_2004))*100

In [ ]:
precip_2005_mcs_fillna = precip_2005_mcs.fillna(0)

cumulative_precip_2005_mcs = precip_2005_mcs_fillna.sum(dim='time', skipna=True)

cumulative_precip_2005 = precip_2005.sum(dim='time', skipna=True)

percentage_2005 = ((cumulative_precip_2005_mcs) / (cumulative_precip_2005))*100

In [ ]:
precip_2006_mcs_fillna = precip_2006_mcs.fillna(0)

cumulative_precip_2006_mcs = precip_2006_mcs_fillna.sum(dim='time', skipna=True)

cumulative_precip_2006 = precip_2006.sum(dim='time', skipna=True)

percentage_2006 = ((cumulative_precip_2006_mcs) / (cumulative_precip_2006))*100

In [ ]:
precip_2007_mcs_fillna = precip_2007_mcs.fillna(0)

cumulative_precip_2007_mcs = precip_2007_mcs_fillna.sum(dim='time', skipna=True)

cumulative_precip_2007 = precip_2007.sum(dim='time', skipna=True)

percentage_2007 = ((cumulative_precip_2007_mcs) / (cumulative_precip_2007))*100

In [ ]:
percenatge_all = xr.concat([percentage_2001, percentage_2002, percentage_2003, percentage_2004, percentage_2005, percentage_2006, percentage_2007], dim='time', compat='override', coords='all')

mean_percentage_all = percenatge_all.mean(dim='time')

mean_percentage_all

In [ ]:
mean_percentage_all.max()

In [ ]:
import matplotlib.patches as mpatches

# Irregular levels to illustrate the use of a proportional colorbar
levels = [0, 10, 20, 30, 40, 50, 60, 70]

fig = plt.figure()

ax1 = plt.subplot(projection=ccrs.PlateCarree())
m = Basemap(llcrnrlon=-85, llcrnrlat=-40,urcrnrlon=-30, urcrnrlat=15) # create Basemap object
m.drawcoastlines(linewidth=2,) # draw coastlines
mean_percentage_all.plot(levels=levels,cmap='Spectral', alpha=0.8, ax=ax1, cbar_kwargs={'ticks': levels, 'label': 'precipitation from MCSs [%]'})
ax1.set_xlim(-85,-30)
ax1.set_title('MCS contribution to total rainfall')
ax1.set_xlabel(' ')
ax1.set_ylabel(' ')

### South East Brazil (52-34^w, 32-14^S) ###
### ------------------------------------ ###
seb = ax1.add_patch(mpatches.Rectangle(xy=[-52, -32], width=18, height=18, linewidth=2,
                                facecolor='none', edgecolor='orange',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.61, 0.45, 'SEB',color='orange',transform=ax1.transAxes,fontsize=f)


### La Plata Basin (72-52^W, 35-18^S) ###
### --------------------------------- ###
lpb = ax1.add_patch(mpatches.Rectangle(xy=[-72, -35], width=20, height=17, linewidth=2,
                                facecolor='none', edgecolor='darkred',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.24, 0.37, 'LPB',color='darkred',transform=ax1.transAxes,fontsize=f)


### Western Amazon (78-65^W, 18-5^S) ###
### -------------------------------- ###
wamz = ax1.add_patch(mpatches.Rectangle(xy=[-78, -18], width=13, height=23, linewidth=2,
                                facecolor='none', edgecolor='crimson',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.14, 0.82, 'WAMZ',color='crimson',transform=ax1.transAxes,fontsize=f)


### Eastern Amazon (65-52^W, 18-5^S) ###
### -------------------------------- ###
eamz = ax1.add_patch(mpatches.Rectangle(xy=[-65, -18], width=13, height=23, linewidth=2,
                                facecolor='none', edgecolor='teal',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.37, 0.82, 'EAMZ',color='teal',transform=ax1.transAxes,fontsize=f)


### North East Brazil (52-34^W, 15-0^S) ###
### ----------------------------------- ###
neb = ax1.add_patch(mpatches.Rectangle(xy=[-52, -14], width=18, height=14, linewidth=2,
                                facecolor='none', edgecolor='darkblue',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.61, 0.72, 'NEB',color='darkblue',transform=ax1.transAxes,fontsize=f)


In [ ]:
f=14
fig = plt.figure(figsize=(12,12))

ax1 = plt.subplot(121, projection=ccrs.PlateCarree())
ax1.set_extent([-85,-30,-40,12], crs=ccrs.PlateCarree())
ax1.coastlines(alpha=0.5)

m = Basemap(llcrnrlon=-85, llcrnrlat=-40,urcrnrlon=-30, urcrnrlat=15) # create Basemap object
m.drawcoastlines(linewidth=1) # draw coastlines
precip = bias_cpm.plot(levels=levels_1, cmap='BrBG', alpha=0.8, ax=ax1, add_colorbar=False)#, cbar_kwargs={'ticks': levels_1,'label': 'average precipitation bias [mm d-1]', 'shrink': 0.4})
ax1.set_xlim(-85,-30)
ax1.set_title('CPM-Hindcast - IMERG', fontsize=14)
ax1.set_xlabel(' ')
ax1.set_ylabel(' ')

### South East Brazil (52-34^w, 32-14^S) ###
### ------------------------------------ ###
seb = ax1.add_patch(mpatches.Rectangle(xy=[-52, -32], width=18, height=18, linewidth=2,
                                facecolor='none', edgecolor='orange',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.61, 0.45, 'SEB',color='orange',transform=ax1.transAxes,fontsize=f)


### La Plata Basin (72-52^W, 35-18^S) ###
### --------------------------------- ###
lpb = ax1.add_patch(mpatches.Rectangle(xy=[-72, -35], width=20, height=17, linewidth=2,
                                facecolor='none', edgecolor='darkred',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.24, 0.37, 'LPB',color='darkred',transform=ax1.transAxes,fontsize=f)


### Western Amazon (78-65^W, 18-5^S) ###
### -------------------------------- ###
wamz = ax1.add_patch(mpatches.Rectangle(xy=[-78, -18], width=13, height=23, linewidth=2,
                                facecolor='none', edgecolor='crimson',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.14, 0.82, 'WAMZ',color='crimson',transform=ax1.transAxes,fontsize=f)


### Eastern Amazon (65-52^W, 18-5^S) ###
### -------------------------------- ###
eamz = ax1.add_patch(mpatches.Rectangle(xy=[-65, -18], width=13, height=23, linewidth=2,
                                facecolor='none', edgecolor='teal',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.37, 0.82, 'EAMZ',color='teal',transform=ax1.transAxes,fontsize=f)


### North East Brazil (52-34^W, 15-0^S) ###
### ----------------------------------- ###
neb = ax1.add_patch(mpatches.Rectangle(xy=[-52, -14], width=18, height=14, linewidth=2,
                                facecolor='none', edgecolor='darkblue',
                                transform=ccrs.PlateCarree()))
#ax1.text(0.61, 0.72, 'NEB',color='darkblue',transform=ax1.transAxes,fontsize=f)